# Detector simulation stage
In this notebook I am simulating the propagation of the electrons in the LArTPC taking into account the recombination (quenching) and the drifting (lifetime and diffusion). Each step is implement as a PyTorch module.

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import astropy.units as u

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
physicalParams = {
    'MeVToElectrons': 4.237e+04,
    'alpha': 0.847,
    'beta': 0.2061
}

tpcParams = {
    'vdrift': 0.153812 * u.cm/u.us,  
    'lifetime': 10e3 * u.us,
    'tpcBorders': ((-150, 150), (-150, 150), (-150, 150)) * u.cm, 
    'longDiff': 6.2e-6 * u.cm*u.cm/u.us, 
    'tranDiff': 16.3e-6 *u.cm
}

## Tracks loading
Here we load some simulated tracks from a text file and we store them in a PyTorch tensor

In [4]:
tracks = pd.read_csv('tracks.txt', delim_whitespace=True)
tracks['pdgID'] = 13
tracks['spillID'] = 1
tracks['runID'] = 1
tracks['subRunID'] = 1
tracks['interactionID'] = 1
tracks['t'] = 0

In [5]:
tracks['dx'] = np.sqrt(pow(tracks['x_end']-tracks['x_start'], 2) + 
                       pow(tracks['y_end']-tracks['y_start'], 2) +
                       pow(tracks['z_end']-tracks['z_start'], 2))
tracks['x'] = (tracks['x_end']+tracks['x_start'])/2
tracks['y'] = (tracks['y_end']+tracks['y_start'])/2
tracks['z'] = (tracks['z_end']+tracks['z_start'])/2 
tracks['dE'] = np.abs(tracks['dE'])*1e3
tracks['dEdx'] = tracks['dE']/tracks['dx']

ix = tracks.columns.get_loc("x")
iy = tracks.columns.get_loc("y")
it = tracks.columns.get_loc("t")
iz = tracks.columns.get_loc("z")
idE = tracks.columns.get_loc("dE")
idx = tracks.columns.get_loc("dx")
idEdx = tracks.columns.get_loc("dEdx")

iNElectrons = len(tracks.columns)
iLongDiff = len(tracks.columns)+1
iTranDiff = len(tracks.columns)+2

print("We have %i track segments" % len(tracks))

We have 4194 track segments


In [6]:
%%timeit
torch.tensor(tracks.values).to(device)

532 µs ± 184 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# I need to repeat this because of %%timeit bug https://github.com/jupyter/notebook/issues/4532
segmentedTracks = torch.tensor(tracks.values).to(device) 

## Quenching stage
This class implements the recombination simulation of the ionized electrons. It adds a new column to the tensor with the number of ionized electrons generated by each track segment. 

In [8]:
class Quenching(torch.nn.Module):
    """
    PyTorch module which implements the quenching of the electrons
    in the TPC. 
    """
    
    def forward(self, x):
        """The number of electrons ionized by the track segment is calculated
        taking into account the recombination.
        
        Returns:
            x: a new tensor with an additional column for the number of ionized
            electrons
        """

        add_columns = torch.nn.ZeroPad2d((0, 1, 0, 0))
        x = add_columns(x)

        recomb = torch.log(physicalParams['alpha'] + physicalParams['beta'] * x[:,idEdx]) / (physicalParams['beta'] * x[:,idEdx])
        recomb = torch.where(recomb<=0, torch.zeros_like(recomb), recomb)
        recomb = torch.where(torch.isnan(recomb), torch.zeros_like(recomb), recomb)
        x[:,iNElectrons] = physicalParams['MeVToElectrons'] * x[:,idE] * recomb
        return x

In [9]:
%%timeit
quenching = Quenching()
quenching(segmentedTracks)

216 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
# I need to repeat this because of %%timeit bug https://github.com/jupyter/notebook/issues/4532
quenching = Quenching()
quenchedTracks = quenching(segmentedTracks)

## Drifting stage
This class implements the drifting simulation of the ionized electrons, taking into account electron lifetime and diffusion. It adds two new columns to the tensor, corresponding to the transverse and longitudinal diffusion coefficients.

In [14]:
class Drifting(torch.nn.Module):
    """
    PyTorch module which implements the propagation of the
    electrons towards the anode. 
    """
    
    def forward(self, x):
        """The z coordinate of the track segment is set to 
        the z coordinate of the anode. 
        The number of electrons is corrected by the electron lifetime. 
        The longitudinal and transverse diffusion factors are calculated.
        The time is set to the drift time, calculated taking into account the transverse 
        diffusion factor
        
        Returns:
            x: a new tensor with 2 additional column, for the longitudinal
            diffusion and transverse diffusion coefficients
        """
        
        add_columns = torch.nn.ZeroPad2d((0, 2, 0, 0))
        x = add_columns(x)
        
        zStart =  tpcParams['tpcBorders'][2][0].value

        driftDistance = torch.abs(x[:,iz] - zStart)
        driftTime = driftDistance / tpcParams['vdrift'].value
        x[:,iz] = zStart
        
        lifetime = torch.exp(-driftTime / tpcParams['lifetime'].value)
        x[:,iNElectrons] = x[:,iNElectrons] * lifetime
        
        x[:,iLongDiff] = torch.sqrt(driftTime) * tpcParams['longDiff'].value
        x[:,iTranDiff] = torch.sqrt(driftTime) * tpcParams['tranDiff'].value
        x[:,it] += driftTime + x[:,iTranDiff] / tpcParams['vdrift'].value


        return x

In [15]:
%%timeit
drifting = Drifting()
drifting(quenchedTracks)

321 µs ± 5.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
# I need to repeat this because of %%timeit bug https://github.com/jupyter/notebook/issues/4532
drifting = Drifting()
driftedTracks = drifting(quenchedTracks)

## Tests
### 3D tracks
This plot shows a 3D scatter plot of the simulated tracks and their drifted position at the anode (z = -150 cm).

In [17]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(10):
    ttrack = segmentedTracks[segmentedTracks[:,0] == i]
    driftedTrack = driftedTracks[driftedTracks[:,0] == i]

    plotTrack = ax.scatter(ttrack[:,ix].cpu().numpy(),
                           ttrack[:,iy].cpu().numpy(),
                           ttrack[:,iz].cpu().numpy())

    plotTrack = ax.scatter(driftedTrack[:,ix].cpu().numpy(),
                           driftedTrack[:,iy].cpu().numpy(),
                           driftedTrack[:,iz].cpu().numpy(), 
                           color=plotTrack.get_facecolor())
    
ax.set_xlim(tpcParams['tpcBorders'][0].value)
ax.set_ylim(tpcParams['tpcBorders'][1].value)
ax.set_zlim(tpcParams['tpcBorders'][2].value)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.set_zlabel("z [cm]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Electron lifetime plot
This plot shows the difference between the electrons after the quenching and the electrons at the anode. The difference is caused by the finite electron lifetime.

In [18]:
fig, ax = plt.subplots(1,1)
# ax.set_xlim(0,2e4)
ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax.set_xlabel(r"$N_{\mathrm{electrons}}^{\mathrm{quenched}}-N_{\mathrm{electrons}}^{\mathrm{drifted}}$")
ax.hist(quenchedTracks[:,iNElectrons].cpu().numpy()-driftedTracks[:,iNElectrons].cpu().numpy(), bins=100, range=(0,4e4))
ax.set_xlim(left=0, right=4e4)
ax.set_ylabel("N. entries / %g" % (4e4/100))
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Transverse diffusion plot
This plot shows that tracks generated at a greater distance from the anode have a larger transverse diffusion coefficient, as expected.

In [19]:
fig, ax = plt.subplots(1,1)
ax.set_ylim(0,0.0010)
ax.set_xlim(tpcParams['tpcBorders'][2][0].value, tpcParams['tpcBorders'][2][1].value)
ax.set_xlabel("Start z [cm]")
ax.set_ylabel("Transverse diffusion coefficient")
_ = ax.scatter(quenchedTracks[:,iz].cpu().numpy(), driftedTracks[:,iTranDiff].cpu().numpy())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …